In [6]:
import speech_recognition as sr
from os import path

In [7]:
wav_file = "D:/projects/speech to text/"

AUDIO_FILE = path.join(wav_file, "kimberly.wav")

In [8]:
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source) # read the entire audio file

# recognize speech using Sphinx
try:
    print("Sphinx thinks you said: " + r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

Sphinx thinks you said: hi i'm tenderly one of the available high-quality text is speech places select download now to install my voice


In [ ]:
# obtain audio from the microphone
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)

# write audio to a WAV file
with open("microphone-results.wav", "wb") as f:
    f.write(audio.get_wav_data())

In [1]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest

In [2]:
class FrequencySummarizer:
    def __init__(self, min_cut=0.1, max_cut=0.9):
        """
         Initilize the text summarizer.
         Words that have a frequency term lower than min_cut 
         or higer than max_cut will be ignored.
        """
        self._min_cut = min_cut
        self._max_cut = max_cut 
        self._stopwords = set(stopwords.words('english') + list(punctuation))

    def _compute_frequencies(self, word_sent):
        """ 
          Compute the frequency of each of word.
          Input: 
           word_sent, a list of sentences already tokenized.
          Output: 
           freq, a dictionary where freq[w] is the frequency of w.
        """
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
        # frequencies normalization and fitering
        m = float(max(freq.values()))
        for w in freq.keys():
            freq[w] = freq[w]/m
            if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
                del freq[w]
        return freq

    def summarize(self, text, n):
        """
          Return a list of n sentences 
          which represent the summary of text.
        """
        sents = sent_tokenize(text)
        assert n <= len(sents)
        word_sent = [word_tokenize(s.lower()) for s in sents]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sent in enumerate(word_sent):
            for w in sent:
                if w in self._freq:
                    ranking[i] += self._freq[w]
        sents_idx = self._rank(ranking, n)    
        return [sents[j] for j in sents_idx]

    def _rank(self, ranking, n):
        """ return the first n sentences with highest ranking """
        return nlargest(n, ranking, key=ranking.get)

In [3]:
import urllib2
from bs4 import BeautifulSoup

def get_only_text(url):
     """ 
      return the title and the text of the article
      at the specified url
     """
     page = urllib2.urlopen(url).read().decode('utf8')
     soup = BeautifulSoup(page)
     text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
     return soup.title.text, text

In [5]:
feed_xml = urllib2.urlopen('http://feeds.bbci.co.uk/news/rss.xml').read()
feed = BeautifulSoup(feed_xml.decode('utf8'))
to_summarize = map(lambda p: p.text, feed.find_all('guid'))

fs = FrequencySummarizer()
for article_url in to_summarize[:5]:
  title, text = get_only_text(article_url)
  print '----------------------------------'
  print title
  for s in fs.summarize(text, 2):
   print '*',s

----------------------------------
Theresa May promises mental health care overhaul - BBC News
* In the speech, Mrs May announced: Mrs May said that mental health had been "dangerously disregarded" as secondary to physical health and changing that will go "right to the heart of our humanity".
* Simon Walker, director general at the Institute of Directors, said: "Mental health problems touch us all, and employers have a real role to play ensuring the health - physical and mental - of their workforce."
----------------------------------
Golden Globes 2017: La La Land sweeps awards - BBC News
* Affleck was named best actor in a film drama for his role in Manchester By The Sea and French star Huppert was the surprise winner of the award for best film drama actress.
* The talk show host joked that the Golden Globes ceremony was "one of the few places left where America still honours the popular vote" - a reference to Donald Trump beating Hillary Clinton in the recent US election despite get